In [1]:
import tkinter as tk
import pandas as pd
import random
from googletrans import Translator, LANGUAGES

# Configuration initiale
PHRASE_FILE = "phrases_annotees.csv"
SAVE_FILE = "phrases_annotees.csv"
TARGET_LANG = 'fr'  # Français

class BinaryClassifier:
    def __init__(self, master):
        self.master = master
        master.title("Classificateur Binaire - Traduction FR")
        
        # Initialisation du traducteur
        self.translator = Translator()
        self.translate_cache = {}
        
        # Chargement des données
        try:
            self.df = pd.read_csv(PHRASE_FILE)
        except FileNotFoundError:
            print(f"Créez d'abord le fichier {PHRASE_FILE} avec une colonne 'phrase'")
            exit()
        
        # Initialisation de la colonne de label si nécessaire
        if 'label' not in self.df.columns:
            self.df['label'] = -1  # -1 = non annoté
        
        # Phrases non annotées
        self.non_annotees = self.df[self.df['label'] == -1].index.tolist()
        random.shuffle(self.non_annotees)
        
        # Interface graphique
        self.frame_phrase = tk.LabelFrame(master, text="Phrase à classer")
        self.frame_phrase.pack(pady=20, padx=20, fill="both", expand=True)
        
        # Phrase originale
        self.label_orig = tk.Label(
            self.frame_phrase, 
            text="", 
            wraplength=900,
            justify="left",
            bg="#F0F0F0",
            font=("Arial", 16)
        )
        self.label_orig.pack(pady=10, padx=10, fill="x")
        
        # Traduction
        self.frame_trad = tk.LabelFrame(self.frame_phrase, text=f"Traduction ({LANGUAGES[TARGET_LANG].capitalize()})")
        self.frame_trad.pack(pady=10, padx=10, fill="x")
        
        self.label_trad = tk.Label(
            self.frame_trad, 
            text="", 
            wraplength=900,
            justify="left",
            bg="#FFFFFF",
            font=("Arial", 15)
        )
        self.label_trad.pack(pady=10, padx=10, fill="x")
        
        # Compteur
        self.compteur_frame = tk.Frame(master)
        self.compteur_frame.pack(pady=5)
        self.label_compteur = tk.Label(
            self.compteur_frame,
            text="",
            font=("Arial", 10)
        )
        self.label_compteur.pack()
        
        # Boutons
        self.btn_frame = tk.Frame(master)
        self.btn_frame.pack(pady=20)
        
        self.btn_positive = tk.Button(
            self.btn_frame, 
            text="Positive (P)", 
            command=lambda: self.enregistrer_label(1),
            bg="#4CAF50",
            fg="white",
            width=15,
            font=("Arial", 10, "bold")
        )
        self.btn_positive.pack(side="left", padx=10)
        
        self.btn_negative = tk.Button(
            self.btn_frame, 
            text="Negative (N)", 
            command=lambda: self.enregistrer_label(0),
            bg="#F44336",
            fg="white",
            width=15,
            font=("Arial", 10, "bold")
        )
        self.btn_negative.pack(side="left", padx=10)
        
        self.btn_quitter = tk.Button(
            master, 
            text="Sauvegarder et Quitter (Q)", 
            command=self.sauvegarder_quitter,
            bg="#607D8B",
            fg="white",
            font=("Arial", 10)
        )
        self.btn_quitter.pack(pady=20)
        
        # Raccourcis clavier
        master.bind("<p>", lambda e: self.enregistrer_label(1))
        master.bind("<n>", lambda e: self.enregistrer_label(0))
        master.bind("<q>", lambda e: self.sauvegarder_quitter())
        
        # Première phrase
        self.nouvelle_phrase()

    def traduire_phrase(self, phrase):
        """Traduit une phrase avec cache pour optimiser"""
        if phrase in self.translate_cache:
            return self.translate_cache[phrase]
        
        try:
            trad = self.translator.translate(phrase, dest=TARGET_LANG).text
            self.translate_cache[phrase] = trad
            return trad
        except Exception as e:
            print(f"Erreur de traduction: {e}")
            return "Échec de traduction"

    def nouvelle_phrase(self):
        """Affiche une nouvelle phrase à classer"""
        if not self.non_annotees:
            self.label_orig.config(text="Toutes les phrases sont annotées !")
            self.label_trad.config(text="")
            self.btn_positive.config(state="disabled")
            self.btn_negative.config(state="disabled")
            self.label_compteur.config(text="Annotation terminée - 0 phrase restante")
            return
            
        self.idx = self.non_annotees.pop(0)
        phrase = self.df.loc[self.idx, 'phrase']
        trad = self.traduire_phrase(phrase)
        
        # Mise à jour de l'interface
        self.label_orig.config(text=phrase)
        self.label_trad.config(text=trad)
        
        # Mise à jour du compteur
        restantes = len(self.non_annotees)
        self.label_compteur.config(
            text=f"Phrases restantes: {restantes} | Dernier label: {self.get_last_label()}"
        )

    def get_last_label(self):
        """Obtient le dernier label enregistré (pour affichage)"""
        if not hasattr(self, 'last_label'):
            return "N/A"
        return "Positive" if self.last_label == 1 else "Negative"

    def enregistrer_label(self, label):
        """Enregistre le label dans le DataFrame"""
        self.df.at[self.idx, 'label'] = label
        self.last_label = label
        
        # Passer à la phrase suivante
        self.nouvelle_phrase()

    def sauvegarder_quitter(self):
        """Sauvegarde les résultats et quitte l'application"""
        self.df.to_csv(SAVE_FILE, index=False)
        self.master.destroy()
        print(f"Annotations sauvegardées dans {SAVE_FILE}")
        print(f"Phrases annotées: {len(self.df) - len(self.non_annotees)}")
        print("Les traductions ne sont PAS sauvegardées dans le fichier final")

# Lancement de l'application
if __name__ == "__main__":
    root = tk.Tk()
    app = BinaryClassifier(root)
    root.mainloop()

Annotations sauvegardées dans phrases_annotees.csv
Phrases annotées: 222
Les traductions ne sont PAS sauvegardées dans le fichier final
